In [29]:
# Import necessary library
from pypdf import PdfReader

import os
from dotenv import load_dotenv
from openai import OpenAI

from IPython.display import display, Markdown
import gradio as gr

In [17]:
# Read the pdf file
linkedin_text = ''
file_name = './me/linkedin.pdf'

reader = PdfReader(file_name)

for page_num in range(len(reader.pages)):
    page = reader.pages[page_num]
    text = page.extract_text()
    if text:
        linkedin_text += text + '\n'

In [ ]:
# Read the summary file
summary_text = ''
file_name = './me/summary.txt'

try:
    with open(file_name, 'r') as f:
        summary_text = f.read()
except FileNotFoundError:
    print('File not found')
except Exception as e:
    print(f'Error: {e}')

In [33]:
# Write a system prompt
name = 'Ed Donner'

# Write a system prompt instructing what to do.
# Add to the prompt the resume and summary text.
system_prompt = f'You are acting as {name}.'
system_prompt = f'You are answering questions on {name} website particularly questions regarding {name} career, background, and experience.'
system_prompt = f'Your responsibility is to represent {name} for interactions on the website as faithfully as possible.'
# system_prompt = f'Be professional and engaging, as if talking to future employer or potential client.'
system_prompt = f'Be cute and playful, as if talking to a friend or an acquaintance.'
system_prompt += f'If you do not know the answer, just say so. Do not invention the answer.'
system_prompt += f'Here is some information about {name} that you may refer to in answering those questions.\n\n'
system_prompt += f'\n\nHere is {name} Linkedin information. \n\n {linkedin_text}\n\n'
system_prompt += f'\n\nSome more information about {name} is here.\n\n {summary_text}\n\n'
system_prompt += f'With this context, chat with the user. Always stay in character as {name}.'


In [24]:
# Set up client
load_dotenv()

client = OpenAI(api_key=os.getenv('DEEPSEEK_API_KEY'), base_url='https://api.deepseek.com/v1')

In [34]:
# Write a chat function for Gradio to interact with the user
def chat(message, history):
    messages = [{'role': 'system', 'content': system_prompt}] + history + [{'role': 'user', 'content': message}]
    response = client.chat.completions.create(
        model='deepseek-chat',
        messages=messages
    )

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type='messages').launch()#share=True)

In [ ]:
# setting up evaluator prompt
evaluator_prompt = f"You are to evaluate whether a response to a question is acceptable."
evaluator_prompt = f"You are given a conversation between Agent and User."
evaluator_prompt += f"The Agent was instructed to represent {name} as faithfully as possible."
evaluator_prompt += f"The agent was also instructed to be cute and playful, as if talking to a friend or an acquaintance."
evaluator_prompt += f"Your task is to evaluate whether the agent response met the instruction."
evaluator_prompt += f"The "